# Install DeepQuantreg from GitHub

In [1]:
!pip3 install git+https://github.com/yicjia/DeepQuantreg.git -q

     |████████████████████████████████| 348kB 18.9MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 


# Import packages

In [2]:
from DeepQuantreg import utils as utils
from DeepQuantreg import deep_quantreg as dq
from DeepQuantreg import hyperparameter_tuning as tune
import pandas as pd

# Load data

In [3]:
url = 'https://raw.githubusercontent.com/yicjia/DeepQuantreg/master/data/'
train_dataset_fp = url+"traindata.csv"
train_df = pd.read_csv(train_dataset_fp)
train_df = utils.organize_data(train_df,time="OT",event="ind",trt="x2")

test_dataset_fp = url+"testdata.csv"
test_df = pd.read_csv(test_dataset_fp)
test_df = utils.organize_data(test_df,time="OT",event="ind",trt="x2")

# Hyperparameter tuning


In [7]:
# Specify your search space 
# Finer gird of search space will give better results but cost more time
# You can change the runtime to GPU to speed up this procedure
# Here is a small search space to save time (~6min)
layers = [2]
nodes=[300]
dropout=[0.3,0.4]
activation = ["sigmoid","relu"]
optimizer = ["adam"]
bsize = [64]
n_epochs = [500]

tune.hyper_tuning(train_df, layers, nodes, dropout, activation, optimizer, bsize, n_epochs, n_cv=5, tau=0.5)

{'activation': 'relu',
 'batch_size': 64,
 'dropout': 0.4,
 'epochs': 500,
 'layers': 2,
 'nodes': 300,
 'optimizer': 'adam'}

# Run DeepQuantreg

In [4]:
result = dq.deep_quantreg(train_df,test_df,layer=2,node=300,n_epoch=200,bsize=64,tau=0.5)

Concordance Index for training dataset: 0.7958445122690179
Concordance Index for test dataset: 0.8038966559299506


In [9]:
# get the predicted quantiles and its prediction interval
pred = result.predQ
lowerPI = result.lower
upperPI = result.upper

In [10]:
# get the prediction performance
print("C-index",result.ci)
print("MMSE",result.mse)
print("Quantile loss",result.ql)

C-index 0.8038966559299506
MMSE 1.774973805226771
Quantile loss 0.49455462067822037
